In [1]:
#import dependencies
import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

In [2]:
path = '../weatherpy/outputs/cities.csv'
cities_for_heatmap_df = pd.read_csv(path)
cities_for_heatmap_df.sample(10)

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement
54,metro,-5.11,105.31,74.80,100,80,5.12,ID,1608823788
218,tuatapere,-46.13,167.68,46.99,3,84,4.00,NZ,1608823798
334,port lincoln,-34.73,135.87,57.72,0,78,14.56,AU,1608823828
402,rybnaya sloboda,55.46,50.14,8.60,75,84,4.47,RU,1608823834
238,lolodorf,3.23,10.73,79.68,28,81,3.80,CM,1608823799
544,altay,47.87,88.12,9.36,48,83,1.50,CN,1608823846
348,pauini,-7.71,-66.98,93.25,100,49,2.66,BR,1608823829
51,banjar,-8.19,114.97,82.40,40,78,1.12,ID,1608823788
478,kemang,-3.35,104.34,75.88,100,94,0.98,ID,1608823841
187,norilsk,69.35,88.20,-41.80,20,64,6.71,RU,1608823796


In [3]:
cities_for_heatmap_df['humidity'].max()

100

In [4]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
ideal_weather_df = pd.DataFrame(cities_for_heatmap_df[(cities_for_heatmap_df['max_temp'] < 70) & (cities_for_heatmap_df['max_temp'] > 60) & (cities_for_heatmap_df['wind_speed'] < 10) & (cities_for_heatmap_df['cloudiness'] == 0)])
ideal_weather_df['hotel_name'] = " "
ideal_weather_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,


In [6]:
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={g_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, must drop row')
        hotel_names.append('n/a')
ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

No hotel within 5000 m of tiznit, must drop row
No hotel within 5000 m of shadegan, must drop row


,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Bredasdorp Square
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,n/a
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Man Thi Tar Hotel Minbu
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Vandenberg Hotel
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,n/a
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,The First Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,WORACHAT BOUTIQUE HOTEL
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,فندق الزيتونة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Shapur Hotel


In [7]:
#testing to see if error is because of lack of hotels w/in 5000 m; increasing to 10000 m, then 20000 m
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={g_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, will try 10000 m range')
        try:
            target_radius = 10000
            target_location = f'{row[1]},{row[2]}'
            final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={g_key}'
            data = requests.get(final_url).json()
            hotel_names.append(f"{data['results'][0]['name']}* 10 km away")
            target_radius = 5000
        except IndexError:
            print(f'No hotel within 10000 m of {row[0]}, will try 20000 m range')
            try:
                target_radius = 20000
                target_location = f'{row[1]},{row[2]}'
                final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={g_key}'
                data = requests.get(final_url).json()
                hotel_names.append(f"{data['results'][0]['name']}* 20 km away")
                target_radius = 5000
            except IndexError:
                print(f'no hotel found within 20000 m of {row[0]}, exiting row')
                continue

hotel_names

No hotel within 5000 m of tiznit, will try 10000 m range
No hotel within 10000 m of tiznit, will try 20000 m range
No hotel within 5000 m of shadegan, will try 10000 m range
No hotel within 10000 m of shadegan, will try 20000 m range


['Bredasdorp Square',
 'Hotel kerdous* 20 km away',
 'Man Thi Tar Hotel Minbu',
 'Vandenberg Hotel',
 'Estuary fishing pier Douragh* 20 km away',
 'The First Hotel',
 'โรงแรมต้นค้อ',
 'فندق الزيتونة',
 'Shapur Hotel']

In [8]:
hotels_df = ideal_weather_df.drop(ideal_weather_df[ideal_weather_df.hotel_name == 'n/a'].index)
hotels_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Bredasdorp Square
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Man Thi Tar Hotel Minbu
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Vandenberg Hotel
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,The First Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,WORACHAT BOUTIQUE HOTEL
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,فندق الزيتونة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Shapur Hotel


In [9]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotel_locations = hotels_df[["lat", "long"]]


marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))